In [57]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record,eval_answer
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())
client_dict=get_client_dict(conf)

print("vllmサーバーの実行コマンド")
print(launch_command(conf))

#client_dict

vllmサーバーの実行コマンド
export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001 --gpu-memory-utilization 0.4 




In [68]:
model_name="microsoft/Phi-3-medium-128k-instruct"
pre_command="""次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

In [65]:
#load dataset
from datasets import load_dataset
ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train",streaming=False)

In [67]:
ds=ds.shuffle()

In [94]:

record=ds[0]

#問題文と回答の取得
question,answer=parse_record(record)

#promptの生成
prompt=pre_command+question
res=ask_llm_prompt(client_dict,model_name,prompt)

#生成されたコードを実際にまわして評価する
explanation_block=res[:res.find("```")]
code_block=extract_code_block(res)

#真偽判定
is_correct_answer=eval_answer(code_block,answer,verbose=False)

generated_prompt=explanation_block+"\n"+code_block
if is_correct_answer:
    record["answer"]=generated_prompt

In [95]:
record

{'index': 1145238,
 'dataset': 'math',
 'question': 'In a rectangular coordinate system, what is the number of units in the distance from the origin to the point $(-15, 8)$?',
 'generated_solution': "The distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ in the Cartesian plane is given by \\\\[d=\\\\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}.\\\\]\nLet's evaluate this using sympy for our two points:\n<llm-code>\nfrom sympy import sqrt\n\n# two points\npoints = [(-15, 8), (0, 0)]\n\n# compute distances\ndistance = [sqrt(abs(points[0][0] - p[0])**2 + abs(points[0][1] - p[1])**2) for p in points[1:]]\n\nprint(distance)\n</llm-code>\n<llm-code-output>\n[17]\n</llm-code-output>\nThe distance is $\\\\boxed{17}$ units.",
 'question_ja': '直交座標系において、原点から点 $(-15, 8)$ までの距離は何単位か。',
 'generated_solution_ja': '2つのポイント間の距離$（x_1、y_1）$ $および$（x_2、y_2）$ $ $は、\\\\ [d = \\\\ sqrt {（x_2-x_1）^2+（y_2-y_1）^2}によって与えられます。。\\\\]\n2つのポイントにSympyを使用してこれを評価しましょう。<llm-code>\nfrom sympy import sqrt\n\n# two points\npoints = [

'29.0\n'